In [1]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import os
import sys
from random import randint
from math import ceil, sqrt
import natsort
import pandas as pd
import random
import pickle
from DataLoader import  DataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
#from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import MobileNet

Using TensorFlow backend.
C:\Anaconda\envs\roadcross\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda\envs\roadcross\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda\envs\roadcross\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda\envs\roadcross\lib\site-packages\tensorflow\python\framework\d

In [2]:
user = 'siddhi'

if user == 'siddhi':
    path_videos = 'C:/RoadCrossingAssistant/Data/Videos/'
    path_labels_csv = 'C:/RoadCrossingAssistant/Data/labels_framewise_csv.csv'
    path_labels_list = 'C:/RoadCrossingAssistant/Data/labels_framewise_list.pkl'
    path_frames = 'C:/RoadCrossingAssistant/Data/Frames/'

elif user == 'yagnesh':
    path_videos = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/'
    path_labels_csv = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.csv'
    path_labels_list = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.pkl'

elif user == 'aws':
    path_videos = '/home/ubuntu/Data/Videos/'
    path_labels_csv = '/home/ubuntu/Data/labels_framewise_csv.csv'
    path_labels_list = '/home/ubuntu/Data/labels_framewise_list.pkl'
    path_frames = '/home/ubuntu/Data/Frames/'

# frame-wise labels array
open_file = open(path_labels_list, "rb")
labels_list = pickle.load(open_file)
open_file.close()

In [7]:
x = np.arange(1, 105)
np.random.shuffle(x)
#np.random.seed(42)
videos_validation = x[:18]
videos_test = x[18: 18+24]
videos_train = x[18+24: ]

# videos_train = [videos[ind] for ind in indices_train]
# videos_test = [videos[ind] for ind in indices_test]

# labels_train_loaded = [labels_list[ind] for ind in indices_train]
# labels_test_loaded = [labels_list[ind] for ind in indices_test]

# print('len of videos_train: ', len(videos_train))
# print('len of videos_test: ', len(videos_test))
# print('len of labels_train_loaded: ', len(labels_train_loaded))
# print('len of labels_test_loaded: ', len(labels_test_loaded))

print(videos_train, len(videos_train))
print(videos_test, len(videos_test))
print(videos_validation, len(videos_validation))

[ 59  50  53  64  90 104  93  48  25  89  33  73  63   9   7  16  67  13
  31  37   1   6  80  91  56  78  75  58  19  54  61  83   8  45  74  60
  23  32  97  30  14   4   2 103 100  82  41  24  26  92  29  76  96  77
  42  87  46   3  35  57  98  70] 62
[22 44 79 52 51 49 15 86 94 85 55 39 69 28 81 11 62 43  5 71 65 95 10 34] 24
[ 40  18  12 101  84  27  36  47  17  72  38 102  66  68  20  21  99  88] 18


In [38]:
filenames_train = []
labels_train = []
filenames_validation = []
labels_validation = []
filenames_test = []
labels_test = []

# videos = [1,2]
# filenames = []
# labels = []

for vid in videos_train:
    folder = path_frames + "video{}/".format(vid)
    frames = glob.glob(folder + 'frame*.jpg')
    frames = natsort.natsorted(frames)
    filenames_train.extend(frames)
    labels_path = path_frames + "video{}/".format(vid) + "labels{}.npy".format(vid)
    labels_array = np.load(labels_path)
    labels_list = list(labels_array)
    labels_train.extend(labels_list)

for vid in videos_test:
    folder = path_frames + "video{}/".format(vid)
    frames = glob.glob(folder + 'frame*.jpg')
    frames = natsort.natsorted(frames)
    filenames_test.extend(frames)
    labels_path = path_frames + "video{}/".format(vid) + "labels{}.npy".format(vid)
    labels_array = np.load(labels_path)
    labels_list = list(labels_array)
    labels_test.extend(labels_list)

for vid in videos_validation:
    folder = path_frames + "video{}/".format(vid)
    frames = glob.glob(folder + 'frame*.jpg')
    frames = natsort.natsorted(frames)
    filenames_validation.extend(frames)
    labels_path = path_frames + "video{}/".format(vid) + "labels{}.npy".format(vid)
    labels_array = np.load(labels_path)
    labels_list = list(labels_array)
    labels_validation.extend(labels_list)



#print(len(filenames_train), len(filenames_validation), len(filenames_test))
#print(len(labels_train), len(labels_validation), len(labels_test))
    

# x = []
# for i in range(16):
#   x.append("frame" + str(i) + ".jpg")


15510 4860 6150
15510 4860 6150


In [32]:
labels_path = path_frames + "video{}/".format(1) + "labels{}.npy".format(1)
labels = np.load(labels_path)
l = list(labels)
print(len(l))

210


In [8]:
# Generators
training_generator = DataGenerator(list_IDs = list_IDs_train, folder_path = path_frames)
validation_generator = DataGenerator(list_IDs = list_IDs_validation, folder_path = path_frames)
testing_generator = DataGenerator(list_IDs = list_IDs_test, folder_path = path_frames)


In [14]:
from tensorflow.keras.layers import Flatten, Dense
import keras
import tensorflow
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers 
from tensorflow.keras.applications import MobileNet

base_model = MobileNet(input_shape = (360, 640, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(2, activation='softmax')(x)

model = tensorflow.keras.models.Model(base_model.input, x)
model.compile(
    loss='binary_crossentropy',
    optimizer=tensorflow.keras.optimizers.Adam(),
    metrics=['accuracy'])

model.summary()

C:\Anaconda\envs\roadcross\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 360, 640, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 361, 641, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 180, 320, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 180, 320, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)

In [7]:
#history = model.fit_generator(train_generator, shuffle='true', epochs=1, verbose=1, batch_size=16)

history = model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=15, verbose=1)

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 1 1 1 0 1 0 0 0 1 0]
[0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 0]
[1 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0]
[1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1]
[1 1 1 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 1]
[0 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 0 0 0 1 1 1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1]
  2/666 [..............................] - ETA: 8:10:52 - loss: 1.4485 - acc: 0.6406 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 

KeyboardInterrupt: 

In [1]:
# import random
# from PIL import Image 

# dim = (1080, 1920)
# X = np.empty((4, *dim, 3))
# y = np.empty((4), dtype=int)

# video_ids = [2,3]

# c = 0

# for vid in video_ids:

#     frame_ids = random.sample(list(range(list_IDs_train[vid])), 2) # generate "no_frames_per_video" random frames
#             #max value of frame given by self.list_ids[vid]
#     labels_temp = np.load(path_frames + "video" + str(vid) + "/labels" + str(vid) + ".npy")

#     for fid in frame_ids:
#         im = Image.open(path_frames + "video" + str(vid) + "/frame" + str(fid) + ".jpg")

#         #X[c, ] = np. asarray(im)

#         d = np.asarray(im)
#         frame_resized = cv2.resize(d, (640, 360), interpolation = cv2.INTER_AREA)

#         print(d.shape, frame_resized.shape)
                
#         y[c] = labels_temp[fid]

#         c = c + 1

# print(c)

# print(X.shape, y.shape)

# print(y)

# print(keras.utils.to_categorical(y, num_classes=2))

In [67]:
print("Evaluate on test data")
results = model.evaluate(testing_generator)
print("test loss, test acc:", results)

print("Evaluate on train data")
results = model.evaluate(training_generator)
print("train loss, trai acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
# print("Generate predictions")
# predictions = model.predict(frames_test)
# print("predictions shape:", predictions.shape)
# print(predictions[:10])




Evaluate on test data
567/567 [==============================] - 83s 147ms/sample - loss: 8.3812 - acc: 0.4550
test loss, test acc: [8.381198916695947, 0.45502645]
Generate predictions
predictions shape: (567, 2)


In [2]:
# from sklearn.metrics import average_precision_score
# average_precision = average_precision_score(np.argmax(y_test, axis = 1), np.argmax(predictions, axis = 1))

# print('Average precision-recall score: {0:0.2f}'.format(
#       average_precision))